## Run this notebook

You can launch this notebook using mybinder, by clicking the button below.

<a href="https://mybinder.org/v2/gh/NASA-IMPACT/veda-docs/HEAD?labpath=example-notebooks/nceo-biomass-statistics.ipynb">
<img src="https://mybinder.org/badge_logo.svg" alt="Binder" title="A cute binder" width="150"/> 
</a>

## Approach

   1. 

## About the Data

[NCEO Aboveground Woody Biomass 2017](https://ceos.org/gst/africa-biomass.html) is map for the year 2017 at 100 m spatial resolution which was developed using a combination of LiDAR, Synthetic Aperture Radar (SAR) and optical based data. Aboveground woody biomass (AGB) plays an key role in the study of the Earth's carbon cycle and response to climate change. It is expressed as dry matter in Mg ha-1 and estimation based on Earth Observation measurements is an effective method for regional scale studies. 

**Important note**: Users of this dataset should keep in mind that the map is a continental-scale dataset, generated using a combination of different remote sensing data types, with a single method for the whole study area. Therefore, users should understand that accuracy may vary for different regions and vegetation types.

## The Case Study - 

TBD

## Querying the STAC API

In [1]:
from pystac_client import Client

In [2]:
# Provide STAC API endpoint
STAC_API_URL = "https://staging-stac.delta-backend.com/"

# Declare collection of interest - NCEO Biomass
collection = "nceo_africa_2017"

Now let's check how many total items are available. 

In [3]:
search = Client.open(STAC_API_URL).search(collections=[collection])
items = list(search.items())
print(f"Found {len(items)} items")

Found 1 items


This makes sense as there is only one item available, a map for 2017. 

In [4]:
# Explore one item to see what it contains
items[0]

ID: AGB_map_2017v0m_COG
"Bounding Box: [-18.273529509559307, -35.054059016911935, 51.86423292864056, 37.73103856358817]"
"proj:bbox: [-18.273529509559307, -35.054059016911935, 51.86423292864056, 37.73103856358817]"
proj:epsg: 4326.0
"proj:shape: [81024.0, 78077.0]"
end_datetime: 2017-12-31T23:59:59+00:00
"proj:geometry: {'type': 'Polygon', 'coordinates': [[[-18.273529509559307, -35.054059016911935], [51.86423292864056, -35.054059016911935], [51.86423292864056, 37.73103856358817], [-18.273529509559307, 37.73103856358817], [-18.273529509559307, -35.054059016911935]]]}"
"proj:transform: [0.0008983152841195214, 0.0, -18.273529509559307, 0.0, -0.0008983152841195214, 37.73103856358817, 0.0, 0.0, 1.0]"
start_datetime: 2017-01-01T00:00:00+00:00
"stac_extensions: ['https://stac-extensions.github.io/projection/v1.0.0/schema.json', 'https://stac-extensions.github.io/raster/v1.1.0/schema.json']"
https://stac-extensions.github.io/projection/v1.0.0/schema.json


Explore through the item's assets. We can see from the data's statistics values that the `min` and `max` values for the observed values range from `0` to `429` Mg ha-1.

In [29]:
## Need to update proj:epsg from float to integer, probably best to do this here. 
## Below doesn't work as 'int' isn't subscriptable

# items[0] = int(items[0]["proj:epsg"])
# items[0]


TypeError: 'int' object is not subscriptable

## Reading and accessing the data

Now that we've explored the dataset through the STAC API, let's read and access the dataset itself. 

In [5]:
# This is a workaround that is planning to move up into stackstac itself

import boto3
import stackstac
import rasterio as rio
import rioxarray

gdal_env = stackstac.DEFAULT_GDAL_ENV.updated(always=dict(AWS_NO_SIGN_REQUEST=True, session=rio.session.AWSSession(boto3.Session())))

In [23]:
da = stackstac.stack(search.get_all_items(), gdal_env=gdal_env)
da

<xarray.DataArray 'stackstac-fd05fed6962e2660494026e69a289bf1' (time: 1,
                                                                band: 1,
                                                                y: 81025,
                                                                x: 78078)>
dask.array<fetch_raster_window, shape=(1, 1, 81025, 78078), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/16)
  * time            (time) datetime64[ns] NaT
    id              (time) <U19 'AGB_map_2017v0m_COG'
  * band            (band) <U11 'cog_default'
  * x               (x) float64 -18.27 -18.27 -18.27 ... 51.86 51.86 51.86
  * y               (y) float64 37.73 37.73 37.73 37.73 ... -35.05 -35.05 -35.05
    proj:bbox       object {37.73103856358817, 51.86423292864056, -35.0540590...
    ...              ...
    proj:transform  object {0.0, 1.0, 37.73103856358817, 0.000898315284119521...
    proj:shape      object {81024.0, 78077.0}
    raster:bands    object {'scale': 1.0, 'nodata': 'inf', 'offset': 0.0, 'sa...
    description     <U47 'Cloud optimized default layer to display on map'
    title           <U17 'Default COG Layer'
    epsg            float64 4.326e+03
Attributes:
    spec:        RasterSpec(epsg=4326.0, bounds=(-18.274427824843425, -35.054...
    crs:         epsg:4326.0
    transform:   | 0.00, 0.00,-18.27|\n| 0.00,-0.00, 37.73|\n| 0.00, 0.00, 1.00|
    resolution:  0.0008983152841195214

In [7]:
# Create an AOI for our study area

# Guinea
guinea_aoi = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
          [
            [
              -15.519958756713947,
              12.732440363049193
            ],
            [
              -15.519958756713947,
              6.771426493209475
            ],
            [
              -7.078554695621165,
              6.771426493209475
            ],
            [
              -7.078554695621165,
              12.732440363049193
            ],
            [
              -15.519958756713947,
              12.732440363049193
            ]
          ]
        ],
        "type": "Polygon"
      }
    }

#TODO: replace with admin 1 or admin 2 boundaries

In [8]:
# Subset to bounding box of Guinea
subset = da.rio.clip([guinea_aoi["geometry"]])
subset

<xarray.DataArray 'stackstac-13b1580b21580ea059e0f45ca824e1ff' (time: 1,
                                                                band: 1,
                                                                y: 6636, x: 9397)>
dask.array<getitem, shape=(1, 1, 6636, 9397), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/17)
  * time            (time) datetime64[ns] NaT
    id              (time) <U19 'AGB_map_2017v0m_COG'
  * band            (band) <U11 'cog_default'
  * x               (x) float64 -15.52 -15.52 -15.52 ... -7.081 -7.08 -7.079
  * y               (y) float64 12.73 12.73 12.73 12.73 ... 6.773 6.772 6.772
    proj:bbox       object {-35.054059016911935, 51.86423292864056, 37.731038...
    ...              ...
    proj:shape      object {81024.0, 78077.0}
    raster:bands    object {'scale': 1.0, 'nodata': 'inf', 'offset': 0.0, 'sa...
    description     <U47 'Cloud optimized default layer to display on map'
    title           <U17 'Default COG Layer'
    epsg            float64 4.326e+03
    spatial_ref     int64 0
Attributes:
    spec:        RasterSpec(epsg=4326.0, bounds=(-18.274427824843425, -35.054...
    resolution:  0.0008983152841195214

In [19]:
# select the band of interest, as there is only one in this dataset we'll select the default
data_band = subset.sel(band="cog_default")
data_band

<xarray.DataArray 'stackstac-13b1580b21580ea059e0f45ca824e1ff' (time: 1,
                                                                y: 6636, x: 9397)>
dask.array<getitem, shape=(1, 6636, 9397), dtype=float64, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/17)
  * time            (time) datetime64[ns] NaT
    id              (time) <U19 'AGB_map_2017v0m_COG'
    band            <U11 'cog_default'
  * x               (x) float64 -15.52 -15.52 -15.52 ... -7.081 -7.08 -7.079
  * y               (y) float64 12.73 12.73 12.73 12.73 ... 6.773 6.772 6.772
    proj:bbox       object {-35.054059016911935, 51.86423292864056, 37.731038...
    ...              ...
    proj:shape      object {81024.0, 78077.0}
    raster:bands    object {'scale': 1.0, 'nodata': 'inf', 'offset': 0.0, 'sa...
    description     <U47 'Cloud optimized default layer to display on map'
    title           <U17 'Default COG Layer'
    epsg            float64 4.326e+03
    spatial_ref     int64 0
Attributes:
    spec:        RasterSpec(epsg=4326.0, bounds=(-18.274427824843425, -35.054...
    resolution:  0.0008983152841195214

In [30]:
#import hvplot.xarray
#data_band.hvplot(x="x", y="y", coastline=True, cmap="viridis")
